In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import joblib

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('2_output/train_2.csv')
val = pd.read_csv('2_output/val_2.csv')

In [3]:
train.drop(columns=["PassengerId", "Cabin", "Name", "Group", "num", "VIP"], inplace=True)
val.drop(columns=["PassengerId", "Cabin", "Name", "Group", "num", "VIP"], inplace=True)

In [4]:
train.head()

HomePlanet  CryoSleep  Destination   Age  RoomService  FoodCourt  \
0      Earth      False  TRAPPIST-1e  28.0          0.0       55.0   
1      Earth      False  TRAPPIST-1e  17.0          0.0     1195.0   
2     Europa       True  55 Cancri e  28.0          0.0        0.0   
3       Mars      False  TRAPPIST-1e  20.0          0.0        2.0   
4     Europa       True  55 Cancri e  36.0          0.0        0.0   

   ShoppingMall    Spa  VRDeck  Transported  Age_group  total_spent  \
0           0.0  656.0     0.0        False          4        711.0   
1          31.0    0.0     0.0        False          2       1226.0   
2           0.0    0.0     0.0         True          4          0.0   
3         289.0  976.0     0.0         True          3       1267.0   
4           0.0    0.0     0.0         True          4          0.0   

   log_total_spent  spent  Group_size  solo deck side  
0         6.568078      1           1     1    B    S  
1         7.112327      1           3     0    F    P  
2         0.000000      0           5     0    C    S  
3         7.145196      1           2     0    F    P  
4         0.000000      0           5     0    C    P

In [5]:
X_train = train.drop('Transported', axis=1).copy()
y_train = train['Transported'].copy().astype(int)

X_val = val.drop('Transported', axis=1).copy()
y_val = val['Transported'].copy().astype(int)

In [6]:
cols = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

for value in cols:
    X_train[value] = np.log(1+X_train[value])
    X_val[value]=np.log(1+X_val[value])

In [7]:
X_train["CryoSleep"] = X_train["CryoSleep"].astype("int")
X_val["CryoSleep"] = X_val["CryoSleep"].astype("int")

X_train["Age_group"] = X_train["Age_group"].astype("int")
X_val["Age_group"] = X_val["Age_group"].astype("int")


In [8]:
deck_dict = {
    "A":1,
    "B":2,
    "C":3,
    "D":4,
    "E":5,
    "F":6,
    "G":7,
    "T":6
}

X_train["deck"] = X_train["deck"].map(deck_dict)
X_val["deck"] = X_val["deck"].map(deck_dict)

In [9]:
X_train = pd.get_dummies(X_train, columns=['HomePlanet', 'Destination', 'side'])
X_val = pd.get_dummies(X_val, columns=['HomePlanet', 'Destination', 'side'])

In [10]:
boolean_columns = X_train.select_dtypes(include='bool')
X_train[boolean_columns.columns] = boolean_columns.astype(int)
X_val[boolean_columns.columns] = boolean_columns.astype(int)

In [11]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.fit_transform(X_val)

Agora temos a base de treino ajustada e a base de validação ajustada. Utilizarei o cross validation k-folds para avaliar os modelos

In [12]:
model = GaussianNB()

cross_val_scores = cross_val_score(model, X_train, y_train, cv=3)
print("Acurácia (cross-validation) na base de treino: ", round(cross_val_scores.mean(), 2))

model.fit(X_train, y_train)
y_pred_val = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print("Acurácia na base de teste: ", round(val_accuracy, 2))

Acurácia (cross-validation) na base de treino:  0.74
Acurácia na base de teste:  0.74


In [13]:
model = RandomForestClassifier()

cross_val_scores = cross_val_score(model, X_train, y_train, cv=3)
print("Acurácia (cross-validation) na base de treino: ", round(cross_val_scores.mean(), 2))

model.fit(X_train, y_train)
y_pred_val = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print("Acurácia na base de teste: ", round(val_accuracy, 2))

Acurácia (cross-validation) na base de treino:  0.79
Acurácia na base de teste:  0.77


In [14]:
model = SVC()

cross_val_scores = cross_val_score(model, X_train_scaled, y_train, cv=3)
print("Acurácia (cross-validation) na base de treino: ", round(cross_val_scores.mean(), 2))

model.fit(X_train_scaled, y_train)
y_pred_val = model.predict(X_val_scaled)
val_accuracy = accuracy_score(y_val, y_pred_val)
print("Acurácia na base de teste: ", round(val_accuracy, 2))

Acurácia (cross-validation) na base de treino:  0.79
Acurácia na base de teste:  0.77


In [15]:
model = DecisionTreeClassifier()

cross_val_scores = cross_val_score(model, X_train, y_train, cv=3)
print("Acurácia (cross-validation) na base de treino: ", round(cross_val_scores.mean(), 2))

model.fit(X_train, y_train)
y_pred_val = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print("Acurácia na base de teste: ", round(val_accuracy, 2))

Acurácia (cross-validation) na base de treino:  0.73
Acurácia na base de teste:  0.71


In [16]:
model = KNeighborsClassifier()

cross_val_scores = cross_val_score(model, X_train_scaled, y_train, cv=3)
print("Acurácia (cross-validation) na base de treino: ", round(cross_val_scores.mean(), 2))

model.fit(X_train_scaled, y_train)
y_pred_val = model.predict(X_val_scaled)
val_accuracy = accuracy_score(y_val, y_pred_val)
print("Acurácia na base de teste: ", round(val_accuracy, 2))

Acurácia (cross-validation) na base de treino:  0.75
Acurácia na base de teste:  0.71


In [17]:
model = GradientBoostingClassifier()

cross_val_scores = cross_val_score(model, X_train, y_train, cv=3)
print("Acurácia (cross-validation) na base de treino: ", round(cross_val_scores.mean(), 2))

model.fit(X_train, y_train)
y_pred_val = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print("Acurácia na base de teste: ", round(val_accuracy, 2))

Acurácia (cross-validation) na base de treino:  0.8
Acurácia na base de teste:  0.78


In [18]:
param_distributions = {
    'n_estimators': randint(100, 1000),           # Número de árvores entre 100 e 1000
    'learning_rate': uniform(0.01, 0.2),          # Taxa de aprendizado entre 0.01 e 0.2
    'max_depth': randint(3, 10),                  # Profundidade máxima entre 3 e 10
    'min_samples_split': randint(2, 20),          # Mínimo de amostras para split
    'min_samples_leaf': randint(1, 10),           # Mínimo de amostras por folha
    'subsample': uniform(0.7, 0.3),               # Subsample entre 0.7 e 1.0
    'max_features': ['auto', 'sqrt', 'log2']      # Número de features selecionadas
}

random_search = RandomizedSearchCV(
    estimator=GradientBoostingClassifier(),
    param_distributions=param_distributions,
    n_iter=50,                    # Número de combinações aleatórias a serem testadas
    scoring='accuracy',           # Métrica de avaliação (pode ser alterada conforme a necessidade)
    cv=3,                         # Validação cruzada com 3 folds
    n_jobs=-1,                    # Usa todos os núcleos do processador
    random_state=42               # Para reprodutibilidade
)


random_search.fit(X_train, y_train)

print("Melhores parâmetros encontrados:", random_search.best_params_)
print("Melhor pontuação encontrada:", random_search.best_score_)

Melhores parâmetros encontrados: {'learning_rate': 0.021616722433639893, 'max_depth': 7, 'max_features': 'log2', 'min_samples_leaf': 6, 'min_samples_split': 3, 'n_estimators': 443, 'subsample': 0.9497327922401264}
Melhor pontuação encontrada: 0.804716709807305


In [19]:
best_gbc = random_search.best_estimator_
best_gbc

GradientBoostingClassifier(learning_rate=0.021616722433639893, max_depth=7,
                           max_features='log2', min_samples_leaf=6,
                           min_samples_split=3, n_estimators=443,
                           subsample=0.9497327922401264)

In [21]:
model = best_gbc

cross_val_scores = cross_val_score(model, X_train, y_train, cv=3)
print("Acurácia (cross-validation) na base de treino: ", round(cross_val_scores.mean(), 2))

model.fit(X_train, y_train)
y_pred_val = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print("Acurácia na base de teste: ", round(val_accuracy, 2))

Acurácia (cross-validation) na base de treino:  0.81
Acurácia na base de teste:  0.78


In [22]:
param_distributions = {
    'n_estimators': randint(100, 1000),              # Número de árvores
    'max_depth': randint(10, 50),                    # Profundidade máxima das árvores
    'min_samples_split': randint(2, 20),             # Mínimo de amostras para dividir um nó
    'min_samples_leaf': randint(1, 10),              # Mínimo de amostras por folha
    'max_features': ['auto', 'sqrt', 'log2'],        # Máximo de features consideradas
    'bootstrap': [True, False]                       # Se usa bootstrap ou não
}

# Inicializando o RandomizedSearchCV
random_search_rf = RandomizedSearchCV(
    estimator=RandomForestClassifier(),
    param_distributions=param_distributions,
    n_iter=50,                    # Número de combinações a testar
    scoring='accuracy',           # Métrica de avaliação
    cv=3,                         # Validação cruzada
    n_jobs=-1,                    # Usa todos os núcleos
    random_state=42               # Para reprodutibilidade
)

# Ajustando o RandomizedSearchCV
random_search_rf.fit(X_train, y_train)

# Acessando os melhores parâmetros
print("Melhores parâmetros encontrados:", random_search_rf.best_params_)
print("Melhor pontuação encontrada:", random_search_rf.best_score_)

Melhores parâmetros encontrados: {'bootstrap': True, 'max_depth': 41, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 848}
Melhor pontuação encontrada: 0.8042853034224905


In [23]:
best_gbc = random_search_rf.best_estimator_
best_gbc

RandomForestClassifier(max_depth=41, min_samples_leaf=5, min_samples_split=8,
                       n_estimators=848)

In [24]:
model = best_gbc

cross_val_scores = cross_val_score(model, X_train, y_train, cv=3)
print("Acurácia (cross-validation) na base de treino: ", round(cross_val_scores.mean(), 2))

model.fit(X_train, y_train)
y_pred_val = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print("Acurácia na base de teste: ", round(val_accuracy, 2))

Acurácia (cross-validation) na base de treino:  0.8
Acurácia na base de teste:  0.79


In [27]:
joblib.dump(model, 'randomforestclassifier.joblib')

['randomforestclassifier.joblib']